In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [180]:
def crawling(add, brand):
    resp = requests.get(add) # 페이지 주소
    soup = BeautifulSoup(resp.text, 'html.parser')
    
    # 편의점 이름
    tag1 = "float-left font-" + brand + " font-weight-bold"
    #"float-left font-cu font-weight-bold"
    get_name = soup.find("small", tag1) # 편의점 이름을 가져오기 위한 small 태그 select
    name = get_name.get_text() 

    # 상품 분류
    get_type = soup.find_all('small', 'float-right font-weight-bold')
    type_ = []
    for u in get_type:
        type_.append(u.get_text())
    size = len(type_)
    
    # 상품 이름
    get_item = soup.find_all('strong') 
    item = []
    for u in get_item:
        item.append(u.get_text())
    item = item[1:size + 1] 

    # 상품 가격 (여기서는 우선 개당 가격을 가져 옴)
    get_price = soup.find_all('span', 'text-muted small')
    price = []
    for u in get_price:
        tmp = u.get_text()[1:-1][0:-1]
        tmp_new = tmp.replace(",", "")
        price.append(int(tmp_new))

    # 행사 유형
    tag2 = 'badge bg-' + brand + ' text-white'
    #badge bg-cu text-white
    get_event_type = soup.find_all('span', tag2)
    event_type = []
    for u in get_event_type:
        event_type.append(u.get_text())

    # 상품 이미지
    get_img = soup.find_all('img', 'prod_img')
    img = []
    for u in get_img:
        img.append(u.get('src'))
    
    return name, item, type_, price, event_type, img

In [105]:
def appendData(data, name, item, type_, price, event_type, img):
    for i in range(len(img)):
        real_price = int(price[i]) * int(event_type[i][0])
        data = data.append({'편의점 명' : name, '상품 명' : item[i], '분류' : type_[i], '개당 가격' :price[i], '행사 유형' : event_type[i], '이미지' : img[i]}, ignore_index = True)
        
    return data

In [187]:
brand_dic = {'cu' : '6', 'gs25' : '9', 'seven' : '9', 'ministop' : '5', 'emart24' : '9'}

In [182]:
def getData(brand):
    data = pd.DataFrame(columns = ('편의점 명', '상품 명', '분류', '개당 가격', '행사 유형', '이미지'))
    for i in range(1, int(brand_dic[brand]) + 1):
        name, item, type_, price, event_type, img = crawling('https://pyony.com/brands/' + brand + '/?page=' + str(i) + '&event_type=&category=&item=100&sort=&q=', brand)
        data = appendData(data, name, item, type_, price, event_type, img)
    
    return data
    

In [183]:
dataCU = getData('cu')

In [185]:
dataGS25 = getData('gs25')

In [121]:
dataCU.to_csv("data.csv", mode = 'a', header = True, encoding = 'ms949') #ipynb과 같은 경로에 csv파일 생성